In [0]:
import feedparser
import requests as rq
import tweepy
import pandas as pd
import textblob


consumer_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
consumer_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
access_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
access_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [0]:


def scrap_news(stock_ticker):
  for e in d.entries:
    print(e.title)
    print(e.published)
    print(e.link)
    print(e.description)
    print("\n") # 2 newlines

def check_sentiment(stock_ticker):
  
  '''  this function, will check the sentiment for a particular busniess based on the stock ticker '''

  

  #Extracting Tweets
  results = []
  search_ticker = '#'+ stock_ticker
  for tweet in tweepy.Cursor (api.search, q = search_ticker , lang = "en").items(): 
      results.append(tweet)

  #Store tweets data in a data-frame
  def tweets_df(results):
      id_list = [tweet.id for tweet  in results]
      data_set = pd.DataFrame(id_list, columns = ["id"])
      data_set["text"] = [tweet.text for tweet in results]
      data_set["created_at"] = [tweet.created_at for tweet in results]
      data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
      data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
      data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
      data_set["user_location"] = [tweet.author.location for tweet in results]
      data_set["Hashtags"] = [tweet.entities.get('hashtags') for tweet in results] 
      return data_set
	
  data_set = tweets_df(results)

  #Sentiment checking
  text = data_set["text"]
  for i in range(0,len(text)):
      textB = TextBlob(text[i])
      sentiment = textB.sentiment.polarity
      data_set.set_value(i, 'Sentiment',sentiment)
      if sentiment <0.00:
          SentimentClass = 'Negative'
          data_set.set_value(i, 'SentimentClass', SentimentClass )
      elif sentiment >0.00:
          SentimentClass = 'Positive'
          data_set.set_value(i, 'SentimentClass', SentimentClass )
      else:
          SentimentClass = 'Neutral'
          data_set.set_value(i, 'SentimentClass', SentimentClass )

  text = data_set["text"]
  temp = []

  #Extracting hash tags
  for i in range(0,len(text)):
    temp.append ([i for i in text[i].split() if i.startswith("#")])
    
  data_set['Tags'] = temp
  #print(data_set.groupby('SentimentClass')['id'].nunique())
  
  a = float(data_set.query('SentimentClass == "Positive"')['id'].agg('count'))
  b = float(data_set.query('SentimentClass == "Negative"')['id'].agg('count'))
  c = float(data_set.query('SentimentClass == "Neutral"')['id'].agg('count'))

  print ("Public opinion for {}: positive {:.2%} Negetive {:.2%} Neutral {:.2%}") .format(stock_ticker, (a/(a+b+c)), (b/(a+b+c)), (c/(a+b+c)))
  print("\n")
  print ("=================================================================================================\n")
  
